# [Module 1.1] 세이지메이커에서 추론 다커 이미지 빌딩

이 노트북은 아래와 같은 작업을 합니다.


- 참고: 
- [Extending our PyTorch containers](https://sagemaker-examples.readthedocs.io/en/latest/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.html)
- [사용자 정의 다커](https://github.com/gonsoomoon-ml/churn-prediction-workshop2/tree/master/BYOC)
- [TorchServe on AWS](https://catalog.us-east-1.prod.workshops.aws/workshops/04eb9f59-6d25-40c5-a828-67df58b85739/en-US)
---



# 1. 다커 이미지 빌당을 위한 베이스 컨테이너 이미지를 가져옴


In [1]:
import os
import sys
import boto3
import sagemaker
from sagemaker import utils
session = sagemaker.Session()

region = session.boto_region_name
client = session.boto_session.client(
    "sts", region_name=region, endpoint_url=utils.sts_regional_endpoint(region)
)
account = client.get_caller_identity()["Account"]


region = session.boto_region_name


registry_uri_inference = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.8.1",
    py_version="py3",
    instance_type="ml.g4dn.xlarge",
    image_scope="inference",
)

ecr_uri_prefix = account + "." + ".".join(registry_uri_inference.split("/")[0].split(".")[1:])

print("registry_uri_inference: \n", registry_uri_inference)
print("ecr_uri_prefix: ", ecr_uri_prefix)



registry_uri_inference: 
 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3
ecr_uri_prefix:  057716757052.dkr.ecr.us-east-1.amazonaws.com


# 2. 다커 이미지 빌딩
- 추론의 다커 이미지 리파지토리 이름을 지정 합니다.

In [2]:
inference_algorithm_name = "ncf-sagemaker-inference"

훈련 및 추론 다커 이미지에 필요한 파리미터의 값을 확인 합니다.

In [3]:
print("\n###### Inference  #####\n ")
print("account: ", account)
print("region: ", region)
print("inference_algorithm_name: ", inference_algorithm_name)
print("ecr_uri_prefix: ", ecr_uri_prefix)
# print("registry_uri_training.split('/')[0].split('.')[0]}: ", registry_uri_training.split('/')[0].split('.')[0])
print("registry_uri_inference: ", registry_uri_inference)


###### Inference  #####
 
account:  057716757052
region:  us-east-1
inference_algorithm_name:  ncf-sagemaker-inference
ecr_uri_prefix:  057716757052.dkr.ecr.us-east-1.amazonaws.com
registry_uri_inference:  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3


## 2.1. Dockerfile 확인

In [4]:
! cat Dockerfile

ARG REGISTRY_URI
FROM ${REGISTRY_URI}

# Defines inference.py as script entrypoint
ENV SAGEMAKER_PROGRAM inference.py




## 2.2. 추론 이미지 빌딩


In [5]:
%%time
! /bin/bash build_push_inference.sh {account} {region} {inference_algorithm_name} {ecr_uri_prefix} {registry_uri_inference.split('/')[0].split('.')[0]} {registry_uri_inference}
# ! /bin/bash ./container-inference/build_push_inference.sh {account} {region} {inference_algorithm_name} {ecr_uri_prefix} {registry_uri_inference.split('/')[0].split('.')[0]} {registry_uri_inference}

/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/2_Inference/sagemaker_inference_container/container-inference
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  12.45MB
Step 1/3 : ARG REGISTRY_URI
Step 2/3 : FROM ${REGISTRY_URI}
 ---> 2e5970c79f06
Step 3/3 : ENV SAGEMAKER_PROGRAM inference.py
 ---> Using cache
 ---> 0cc97ca7e8de
Successfully built 0cc97ca7e8

# 3. Docker Image 관련 변수 저장

In [6]:
inference_docker_image = f"{ecr_uri_prefix}/{inference_algorithm_name}"
print("inference_docker_image: \n", inference_docker_image)

inference_docker_image: 
 057716757052.dkr.ecr.us-east-1.amazonaws.com/ncf-sagemaker-inference


In [7]:
%store inference_docker_image

Stored 'inference_docker_image' (str)
